# Group 12
## Lab 0

In [1]:
# Import all libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 0.2 Task
### 0.2.1 Transfer Learning from ImageNet  
### Results  
CIFAR-10 data with alexnet (Fine tuning): Accuracy 37.20%   
CIFAR-10 data with alexnet (Feature extraction): Accuracy 76.10%t

In [2]:
# Should return the best performing model after training
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_val_loss = float('inf')
    best_epoch = 0
    best_model = {}
    val_prediction = None
    train_prediction = None
    train_correct = 0
    train_total = 0
    val_correct = 0
    val_total = 0
    train_accuracy = 0.0
    val_accuracy = 0.0
    val_accuracies = []
    train_accuracies = []
    val_losses = []
    train_losses = []
    val_loss_per_epoch = 0.0
    train_loss_per_epoch = 0.0

    # initialize the tensorboard writer
    writer = SummaryWriter()

   
    for epoch in range(num_epochs):

        # tarin
        model.train() # set model in taining mode
        total_train_loss = 0.0
        #for i, data in enumerate(train_loader):
        #    inputs, label = data
        for inputs, label in train_loader:
        
            # optimizer zero grad
            optimizer.zero_grad()

            # model output
            outputs = model(inputs)

            # loss function
            loss = criterion(outputs, label)

            # back propogate
            loss.backward()

            # apply optimser, update the weights
            optimizer.step()

            # accumulate the loss
            total_train_loss += loss.item() * inputs.size(0)

            # Prepare for calculating train accuracy
            _, train_prediction = torch.max(outputs, 1)
            train_total += label.size(0)
            train_correct += (train_prediction == label).sum().item()
            print(f'epoch: {epoch+1}, train_total: {train_total}')

        # Calculate train accuracy
        train_accuracy = train_correct/train_total
        train_accuracies.append(train_accuracy)

        # store all validaion losses for each epoch
        train_loss_per_epoch = total_train_loss / len(train_loader.dataset)
        train_losses.append(train_loss_per_epoch)
        
        # Log training loss to the tensorboard
        writer.add_scalar('training_loss', train_loss_per_epoch, epoch)


        # validate
        model.eval()  # Set the model to evaluation mode
        total_val_loss = 0.0
        for inputs, label in val_loader:
            # apply model
            outputs = model(inputs)

            # calulate loss
            val_loss = criterion(outputs, label)

            # when best model , save it
            if val_loss < best_val_loss:
                # save the model
                best_val_loss = val_loss
                best_epoch = epoch
                best_model = deepcopy(model.state_dict())

            # accumulate loss
            total_val_loss += val_loss.item() * inputs.size(0)

            # Prepare for calculating validation accuracy
            _, val_prediction = torch.max(outputs, 1)
            val_total += label.size(0)
            val_correct += (val_prediction == label).sum().item()
            print(f'epoch: {epoch+1}, val_total: {val_total}')

        # Calculate validation accuracy
        val_accuracy = val_correct / val_total
        val_accuracies.append(val_accuracy)

        # store all validaion losses for each epoch
        val_loss_per_epoch = total_val_loss/len(val_loader.dataset)
        val_losses.append(val_loss_per_epoch)

        # Log training loss to the tensorboard
        writer.add_scalar('validation_loss', val_loss_per_epoch, epoch)

        # Print training and validation loss for each epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss_per_epoch:.4f}, Validation Loss: {val_loss_per_epoch:.4f}")
    
    # calculate average accuracy 
    train_accuracy_final = sum(train_accuracies)/len(train_accuracies)
    print(f'\nTraining Accuracy : {train_accuracy_final * 100:.2f}% ')

    val_accuracy_final = sum(val_accuracies)/len(val_accuracies)
    print(f'\nValidation Accuracy : {val_accuracy_final * 100:.2f}% ')
    
    #print (val_losses)
    #print(num_epochs)
    """
    # Plot
    # 1. test and validation loss vs epoch
    epochs_as_list = []
    epochs_as_list = range(1,num_epochs + 1)
    plt.figure(figsize=(10,6))
    plt.plot(epochs_as_list,val_losses,color='red',linestyle='-',label='Validation Loss')
    plt.plot(epochs_as_list, train_losses, color='green', linestyle='-',label='Training Loss')
    plt.legend()
    plt.xlabel('epoch')
    plt.title('Loss at each epoch')
    plt.show()

    # 2. test and validation accuracy vs epoch
    plt.figure(figsize=(10,6))
    plt.plot(epochs_as_list,val_accuracies,color='red',linestyle='-',label='Validation Accuracy')
    plt.plot(epochs_as_list, train_accuracies, color='green', linestyle='-',label='Training Accuracy')
    plt.legend()
    plt.xlabel('epoch')
    plt.title('Accuracy at each epoch')
    plt.show()
    """
    # Load the best model and return
    model.load_state_dict(best_model)
    
    # close writer
    writer.close()
    
    return model


def test_model(model, test_loader):
    # test with test data on he tarined model
    test_correct = 0
    test_total = 0
    test_accuracy = 0.0
    test_prediction = None
    # initialize the tensorboard writer
    writer_test = SummaryWriter()

    with torch.no_grad():
        pre_list = []
        labels_list = []
        for inputs, label in test_loader:
            # predict
            outputs = model(inputs)
            _, test_prediction = torch.max(outputs, 1)
            # accuracy
            test_correct += (test_prediction == label).sum().item()
            test_total += label.size(0)
            pre_list.extend(test_prediction.tolist())
            labels_list.extend(label.tolist())

    # Plot confusion matrix
    """
    conf_mat = confusion_matrix(pre_list, labels_list)
    conf_mat_visualise = ConfusionMatrixDisplay(conf_mat, display_labels=['Anchor','Ant'])
    #conf_mat_visualise.plot()
    conf_mat_visualise.plot(colorbar=False)
    plt.show()
    """
    
    # Test accuracy
    test_accuracy = test_correct / test_total
    print(f'Test Accuracy {test_accuracy * 100:.2f}% ')
    # Log test accuracy to the tensorboard
    writer_test.add_scalar('Accuracy', test_accuracy)

    # close writer
    writer_test.close()
    
    return test_accuracy


### 
Fine tuning using AlexNet model for CIFAR-10 data

In [3]:
import torchvision.models as models

model_ft = models.alexnet(pretrained=True)

# get the input features of the last layer
in_last_layer = model_ft.classifier[6].in_features

# add a output layer to the model
model_ft.classifier[6] = nn.Linear(in_last_layer, 10)

# Print input size
#print(model_ft)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(len(classes))

10


In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),      # Resize to 224x224, Alexnet expects inputs to be in this size
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# load CIFAR10 data
batch_size = 100 
trainset = torchvision.datasets.CIFAR10(root='./data',train = True, download=True, transform = transform)


# Since the test set has 50000 images, its quite large for training, hence just selecting a small subset (1000 images per class)
# Choose a fixed number of samples per class
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_samples_per_class = 1000 
expected_sample_cnt = len(classes) * num_samples_per_class
#print(expected_sample_cnt)


# Separate the data by class... Break the loop as soon as we have enough samples (as mentioned in num_samples_per_class ) per class
continue_loop = 'j'
class_data = [[] for _ in range(10)]
for idx, (image, label) in enumerate(trainset):
    if (continue_loop == 'j') and (len(class_data[label]) < num_samples_per_class):
        class_data[label].append(idx)
    
    continue_loop = 'n'
    for i in class_data:
        if len(i) < num_samples_per_class:
            continue_loop = 'j'
            break

    if continue_loop == 'n':
        break
    #print(f' {label} length = {len(class_data[label])}')



# Select samples from each class
selected_indices = []
for data in class_data:
    selected_indices.extend(np.random.choice(data, num_samples_per_class, replace=False))
    #print(selected_indices)
    if len(selected_indices) > expected_sample_cnt:
        break

# Create a training subset dataset with balanced classes
trainset_subset = torch.utils.data.Subset(trainset, selected_indices)
train_loader = torch.utils.data.DataLoader(trainset_subset,shuffle = True, batch_size = batch_size)

# Load Test data
testdata = torchvision.datasets.CIFAR10(root='./data',train = False,download = True, transform = transform)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
# Split test data into test set (1000 samples) and validation set (1000 samples)
testset, valset, _ = torch.utils.data.random_split(testdata,[0.1, 0.1,0.8])
test_loader = torch.utils.data.DataLoader(testset,shuffle = True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(valset,shuffle = True, batch_size=batch_size)

In [7]:
print(len(trainset_subset))
print(len(testset))

10000
1000


In [8]:
# set the hyperparameters, loss function and criterion
epochs_ft = 2
LEARNING_RATE_ft = 0.001
criterion_ft = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.Adam(model_ft.parameters(),lr=LEARNING_RATE_ft)

# Train the model
trained_model_ft = train_model(model_ft, criterion_ft, optimizer_ft, train_loader, val_loader, epochs_ft)

# Test the model
tested_model = test_model(trained_model_ft,test_loader)


epoch: 1, train_total: 100
epoch: 1, train_total: 200
epoch: 1, train_total: 300
epoch: 1, train_total: 400
epoch: 1, train_total: 500
epoch: 1, train_total: 600
epoch: 1, train_total: 700
epoch: 1, train_total: 800
epoch: 1, train_total: 900
epoch: 1, train_total: 1000
epoch: 1, train_total: 1100
epoch: 1, train_total: 1200
epoch: 1, train_total: 1300
epoch: 1, train_total: 1400
epoch: 1, train_total: 1500
epoch: 1, train_total: 1600
epoch: 1, train_total: 1700
epoch: 1, train_total: 1800
epoch: 1, train_total: 1900
epoch: 1, train_total: 2000
epoch: 1, train_total: 2100
epoch: 1, train_total: 2200
epoch: 1, train_total: 2300
epoch: 1, train_total: 2400
epoch: 1, train_total: 2500
epoch: 1, train_total: 2600
epoch: 1, train_total: 2700
epoch: 1, train_total: 2800
epoch: 1, train_total: 2900
epoch: 1, train_total: 3000
epoch: 1, train_total: 3100
epoch: 1, train_total: 3200
epoch: 1, train_total: 3300
epoch: 1, train_total: 3400
epoch: 1, train_total: 3500
epoch: 1, train_total: 3600
e

### Feature extraction.

Only the parameters of the final layer is allowed to be updated

In [9]:
import torchvision.models as models

model_fe = models.alexnet(pretrained=True)

# get the input features of the last layer
in_last_layer = model_fe.classifier[6].in_features

# add a output layer to the model
model_fe.classifier[6] = nn.Linear(in_last_layer, 10)

# Print input size
#print(model_fe)

# freeze the parameter update in all the layers
for parameters in model_fe.parameters():
    parameters.requires_grad = False

# unfreeze the parameter update of the last layer
model_fe.classifier[6].weight.requires_grad = True
model_fe.classifier[6].bias.requires_grad = True

In [10]:
# set the hyperparameters, loss function and criterion
epochs_fe = 2
LEARNING_RATE_fe = 0.001
criterion_fe = nn.CrossEntropyLoss()
optimizer_fe = torch.optim.Adam(model_fe.parameters(),lr=LEARNING_RATE_fe)

# Train the model
trained_model_fe = train_model(model_fe, criterion_fe, optimizer_fe, train_loader, val_loader, epochs_fe)

# Test the model
tested_model = test_model(trained_model_fe,test_loader)

epoch: 1, train_total: 100
epoch: 1, train_total: 200
epoch: 1, train_total: 300
epoch: 1, train_total: 400
epoch: 1, train_total: 500
epoch: 1, train_total: 600
epoch: 1, train_total: 700
epoch: 1, train_total: 800
epoch: 1, train_total: 900
epoch: 1, train_total: 1000
epoch: 1, train_total: 1100
epoch: 1, train_total: 1200
epoch: 1, train_total: 1300
epoch: 1, train_total: 1400
epoch: 1, train_total: 1500
epoch: 1, train_total: 1600
epoch: 1, train_total: 1700
epoch: 1, train_total: 1800
epoch: 1, train_total: 1900
epoch: 1, train_total: 2000
epoch: 1, train_total: 2100
epoch: 1, train_total: 2200
epoch: 1, train_total: 2300
epoch: 1, train_total: 2400
epoch: 1, train_total: 2500
epoch: 1, train_total: 2600
epoch: 1, train_total: 2700
epoch: 1, train_total: 2800
epoch: 1, train_total: 2900
epoch: 1, train_total: 3000
epoch: 1, train_total: 3100
epoch: 1, train_total: 3200
epoch: 1, train_total: 3300
epoch: 1, train_total: 3400
epoch: 1, train_total: 3500
epoch: 1, train_total: 3600
e

## 0.2.2 Transfer Learning from MNIST

### Prepare a CNN of your choice and train it on the MNIST data. Report the accuracy

Accuracy 93.50% 

In [11]:
# Define convolution neural network for 1 channel images
import torch.nn as nn
import torch.nn.functional as F

class Net_1Channel(nn.Module):
    def __init__(self):
        super().__init__()
        # Define convolutional layer 1
        self.conv1 = nn.Conv2d(1,6,3) # 1 input channels, 6 output channels, 3x3 kernel size
        
        # Define pooling layer
        self.pool = nn.MaxPool2d(2,2) # 2x2 max pooling with a stride of 2
        
        # Define convolutional layer 1
        self.conv2 = nn.Conv2d(6,16,3)  # 6 input channels, 16 output channels, 3x3 kernel size

        # Define fully connected layers
        """
        The input image size is 28x28 for MNIST.
        After the first convolutional layer (conv1) with a 3x3 kernel and no padding, the spatial dimensions will be reduced to 26x26 (assuming stride = 1).
        After the max pooling layer (pool) with a 2x2 kernel and stride of 2, the spatial dimensions will be reduced to 13x13.
        After the second convolutional layer (conv2) with a 3x3 kernel and no padding, the spatial dimensions will be reduced to 11x11.
        After the second max pooling layer (pool) with a 2x2 kernel and stride of 2, the spatial dimensions will be reduced to 5x5.
        Therefore, the input size for the first fully connected layer (fc1) should be 16 * 5 * 5:
        """
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 16*5*5 input features, 120 output features.. Input must be calulcated carefully based on the o/p of conv2
        self.fc2 = nn.Linear(120, 84) # 120 input features, 84 output features
        self.fc3 = nn.Linear(84, 10)  # 84 input features, 10 output features

    """
        # Define Leaky ReLU activation function
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.1)


    def test(self, x):
        x = self.pool(self.leaky_relu(self.conv1(x)))
        x = self.pool(self.leaky_relu(self.conv2(x)))
        x = torch.flatten(x,1) # flatten all dimensions except batch
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x
    """
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Download the data, preprocess it and create dataloaders

In [12]:
# data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

batch_size = 64

# Load the trainset
trainset = torchvision.datasets.MNIST(root='./data',train = True, download=True, transform = transform)

# Since the test set has 60000 images, its quite large for training, hence just selecting a small subset (1000 images per class)
# Choose a fixed number of samples per class
classes = ('0', '1', '2', '3','4', '5', '6', '7', '8', '9')
num_samples_per_class = 1000 
expected_sample_cnt = len(classes) * num_samples_per_class
#print(expected_sample_cnt)


# Separate the data by class... Break the loop as soon as we have enough samples (as mentioned in num_samples_per_class ) per class
continue_loop = 'j'
class_data = [[] for _ in range(len(classes))]
for idx, (image, label) in enumerate(trainset):
    if (continue_loop == 'j') and (len(class_data[label]) < num_samples_per_class):
        class_data[label].append(idx)
    
    continue_loop = 'n'
    for i in class_data:
        if len(i) < num_samples_per_class:
            continue_loop = 'j'
            break

    if continue_loop == 'n':
        break
    #print(f' {label} length = {len(class_data[label])}')



# Select samples from each class
selected_indices = []
for data in class_data:
    selected_indices.extend(np.random.choice(data, num_samples_per_class, replace=False))
    #print(selected_indices)
    if len(selected_indices) > expected_sample_cnt:
        break

# Create a subset dataset with balanced classes and used to create train loader
trainset_subset = torch.utils.data.Subset(trainset, selected_indices)
train_loader = torch.utils.data.DataLoader(trainset_subset,shuffle = True, batch_size = batch_size)

# download test data
testdata = torchvision.datasets.MNIST(root='./data',train = False,download = True, transform = transform)

# Split the test data in test set (1000 samples) and validation set(1000 samples)
testset, valset, _ = torch.utils.data.random_split(testdata,[0.1, 0.1,0.8])
test_loader = torch.utils.data.DataLoader(testset,shuffle = True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(valset,shuffle = True, batch_size=batch_size)

In [13]:
print(len(trainset_subset))
print(len(testset))

10000
1000


### Train the CNN model on the MNIST dataset

In [23]:
# Define the model
model_cnn_mnist = Net_1Channel()

# Set optimiser and loss function
optimizer_cnn_mnist = optim.Adam(model_cnn_mnist.parameters(),lr=0.001)
criterion_cnn_mnist = nn.CrossEntropyLoss()

# Set the number of epochs. Used a smal number as it takes a lot of training time.
epochs_cnn_mnist = 2

# # Train the model
trained_model_cnn_mnist = train_model(model_cnn_mnist, criterion_cnn_mnist, optimizer_cnn_mnist, train_loader, val_loader, epochs_cnn_mnist)

# save the trained best model
PATH = './mnist_net_cnn.pth'
torch.save(trained_model_cnn_mnist.state_dict(), PATH)


epoch: 1, train_total: 64
epoch: 1, train_total: 128
epoch: 1, train_total: 192
epoch: 1, train_total: 256
epoch: 1, train_total: 320
epoch: 1, train_total: 384
epoch: 1, train_total: 448
epoch: 1, train_total: 512
epoch: 1, train_total: 576
epoch: 1, train_total: 640
epoch: 1, train_total: 704
epoch: 1, train_total: 768
epoch: 1, train_total: 832
epoch: 1, train_total: 896
epoch: 1, train_total: 960
epoch: 1, train_total: 1024
epoch: 1, train_total: 1088
epoch: 1, train_total: 1152
epoch: 1, train_total: 1216
epoch: 1, train_total: 1280
epoch: 1, train_total: 1344
epoch: 1, train_total: 1408
epoch: 1, train_total: 1472
epoch: 1, train_total: 1536
epoch: 1, train_total: 1600
epoch: 1, train_total: 1664
epoch: 1, train_total: 1728
epoch: 1, train_total: 1792
epoch: 1, train_total: 1856
epoch: 1, train_total: 1920
epoch: 1, train_total: 1984
epoch: 1, train_total: 2048
epoch: 1, train_total: 2112
epoch: 1, train_total: 2176
epoch: 1, train_total: 2240
epoch: 1, train_total: 2304
epoch: 1

### Test the CNN model on the MNIST dataset

In [24]:
# Test the model
PATH = './mnist_net_cnn.pth'
trained_model_cnn_mnist.load_state_dict(torch.load(PATH))
tested_model = test_model(trained_model_cnn_mnist,test_loader)


Test Accuracy 94.60% 


## Use the above model as a pre-trained CNN for the SVHN dataset. Report the accuracy

Test Accuracy 70.31% 

### Download the data, preprocess it and create dataloaders

In [25]:
# data augmentation
transform = transforms.Compose([
    transforms.Grayscale(),  # The earier model was trained on MNIST data which was 1 channel gray scale and SVHN is 3 channel rgb so had to convert rgb to gray
    transforms.Resize((28, 28)), # Mnist was 28 * 28 so earlier trained model expects input 28 * 28
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,)) # as its gray scale we need to normalise only 1 channel
])

batch_size = 64

# Load SVHN data
trainset = torchvision.datasets.SVHN(root='./data',split = 'train', download=True, transform = transform)

# Since the test set has 600000 images, its quite large for training, hence just selecting a small subset (1000 images per class)
# Choose a fixed number of samples per class
classes = ('1', '2', '3','4', '5', '6', '7', '8', '9','10')
num_samples_per_class = 1000 
expected_sample_cnt = len(classes) * num_samples_per_class
#print(expected_sample_cnt)


# Separate the data by class... Break the loop as soon as we have enough samples (as mentioned in num_samples_per_class ) per class
continue_loop = 'j'
class_data = [[] for _ in range(len(classes))]
for idx, (image, label) in enumerate(trainset):
    if (continue_loop == 'j') and (len(class_data[label]) < num_samples_per_class):
        class_data[label].append(idx)
    
    continue_loop = 'n'
    for i in class_data:
        if len(i) < num_samples_per_class:
            continue_loop = 'j'
            break

    if continue_loop == 'n':
        break
    #print(f' {label} length = {len(class_data[label])}')



# Select samples from each class
selected_indices = []
for data in class_data:
    selected_indices.extend(np.random.choice(data, num_samples_per_class, replace=False))
    #print(selected_indices)
    if len(selected_indices) > expected_sample_cnt:
        break

# Create a subset of train dataset with balanced classes and use it to create train loader
trainset_subset = torch.utils.data.Subset(trainset, selected_indices)
train_loader = torch.utils.data.DataLoader(trainset_subset,shuffle = True, batch_size = batch_size)

# download test data
testdata = torchvision.datasets.SVHN(root='./data',split = 'test',download = True, transform = transform)

# Split test data into testset (2604 samples) and validation set (2604 samples)
testset, valset, _ = torch.utils.data.random_split(testdata,[0.1, 0.1,0.8])
test_loader = torch.utils.data.DataLoader(testset,shuffle = True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(valset,shuffle = True, batch_size=batch_size)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [26]:
print(len(trainset_subset))
print(len(testset))

10000
2604


In [29]:
# Define the model
model_svhn_ft = Net_1Channel()

# Load the model
PATH = './mnist_net_cnn.pth'
model_svhn_ft.load_state_dict(torch.load(PATH))

# No need the change the last layer as the last layer of the trained model is already having 10 outputs


# set the loss function, criterion and other hyper parameters
epochs_svhn_ft = 2
LEARNING_RATE_svhn_ft = 0.001
criterion_svhn_ft = nn.CrossEntropyLoss()
optimizer_svhn_ft = torch.optim.Adam(model_svhn_ft.parameters(),lr=LEARNING_RATE_svhn_ft)


# Train the model
trained_model_svhn_ft = train_model(model_svhn_ft, criterion_svhn_ft, optimizer_svhn_ft, train_loader, val_loader, epochs_svhn_ft)


# save the trained best model
PATH = './model_svhn_ft.pth'
torch.save(trained_model_svhn_ft.state_dict(), PATH)



epoch: 1, train_total: 64
epoch: 1, train_total: 128
epoch: 1, train_total: 192
epoch: 1, train_total: 256
epoch: 1, train_total: 320
epoch: 1, train_total: 384
epoch: 1, train_total: 448
epoch: 1, train_total: 512
epoch: 1, train_total: 576
epoch: 1, train_total: 640
epoch: 1, train_total: 704
epoch: 1, train_total: 768
epoch: 1, train_total: 832
epoch: 1, train_total: 896
epoch: 1, train_total: 960
epoch: 1, train_total: 1024
epoch: 1, train_total: 1088
epoch: 1, train_total: 1152
epoch: 1, train_total: 1216
epoch: 1, train_total: 1280
epoch: 1, train_total: 1344
epoch: 1, train_total: 1408
epoch: 1, train_total: 1472
epoch: 1, train_total: 1536
epoch: 1, train_total: 1600
epoch: 1, train_total: 1664
epoch: 1, train_total: 1728
epoch: 1, train_total: 1792
epoch: 1, train_total: 1856
epoch: 1, train_total: 1920
epoch: 1, train_total: 1984
epoch: 1, train_total: 2048
epoch: 1, train_total: 2112
epoch: 1, train_total: 2176
epoch: 1, train_total: 2240
epoch: 1, train_total: 2304
epoch: 1

In [30]:
# Test the model
PATH = './model_svhn_ft.pth'
trained_model_svhn_ft.load_state_dict(torch.load(PATH))
tested_model = test_model(trained_model_svhn_ft,test_loader)

Test Accuracy 70.51% 


## In the third step you are performing transfer learning from MNIST to SVHN (optional).

Accuracy: 20.89%

In [31]:
# Define the model
model_svhn_fe = Net_1Channel()

# Load the CNN model that was trained on mnist data set
PATH = './mnist_net_cnn.pth'
model_svhn_fe.load_state_dict(torch.load(PATH))

# no need the change the last layer as the last layer of the trained model is already having 10 outputs

# print model
print(model_svhn_fe)

Net_1Channel(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [32]:
# Freeze the gradient update in all the layers
for parameters in model_svhn_fe.parameters():
    parameters.requires_grad = False

# unfrezee the gradient update in the last layer
model_svhn_fe.fc3.weight.requires_grad = True
model_svhn_fe.fc3.bias.requires_grad = True

In [35]:
# set the loss function, criterion and other hyperparameters
epochs_svhn_fe = 2
LEARNING_RATE_svhn_fe = 0.001
criterion_svhn_fe = nn.CrossEntropyLoss()
optimizer_svhn_fe = torch.optim.Adam(model_svhn_fe.parameters(),lr=LEARNING_RATE_svhn_fe)


# Train the model
trained_model_svhn_fe = train_model(model_svhn_fe, criterion_svhn_fe, optimizer_svhn_fe, train_loader, val_loader, epochs_svhn_fe)


# save the trained best model
PATH = './model_svhn_fe.pth'
torch.save(trained_model_svhn_fe.state_dict(), PATH)

epoch: 1, train_total: 64
epoch: 1, train_total: 128
epoch: 1, train_total: 192
epoch: 1, train_total: 256
epoch: 1, train_total: 320
epoch: 1, train_total: 384
epoch: 1, train_total: 448
epoch: 1, train_total: 512
epoch: 1, train_total: 576
epoch: 1, train_total: 640
epoch: 1, train_total: 704
epoch: 1, train_total: 768
epoch: 1, train_total: 832
epoch: 1, train_total: 896
epoch: 1, train_total: 960
epoch: 1, train_total: 1024
epoch: 1, train_total: 1088
epoch: 1, train_total: 1152
epoch: 1, train_total: 1216
epoch: 1, train_total: 1280
epoch: 1, train_total: 1344
epoch: 1, train_total: 1408
epoch: 1, train_total: 1472
epoch: 1, train_total: 1536
epoch: 1, train_total: 1600
epoch: 1, train_total: 1664
epoch: 1, train_total: 1728
epoch: 1, train_total: 1792
epoch: 1, train_total: 1856
epoch: 1, train_total: 1920
epoch: 1, train_total: 1984
epoch: 1, train_total: 2048
epoch: 1, train_total: 2112
epoch: 1, train_total: 2176
epoch: 1, train_total: 2240
epoch: 1, train_total: 2304
epoch: 1

In [36]:
# Test the model
PATH = './model_svhn_fe.pth'
trained_model_svhn_fe.load_state_dict(torch.load(PATH))
tested_model = test_model(trained_model_svhn_fe,test_loader)

Test Accuracy 22.27% 
